## Mini Project III

In [1]:
import seaborn as sns
import numpy as np
import pandas as pd

import gc 

import plotly.express as px

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering

import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib.pyplot import *

In [3]:
twm_transactions = pd.read_csv("twm_transactions.csv", sep=';')
twm_savings_tran = pd.read_csv("twm_savings_tran.csv", sep=';')
twm_savings_acct = pd.read_csv("twm_savings_acct.csv", sep=';')
twm_customer = pd.read_csv("twm_customer.csv", sep=';')
twm_credit_tran = pd.read_csv("twm_credit_tran.csv", sep=';')
twm_credit_acct = pd.read_csv("twm_credit_acct.csv", sep=';')
twm_checking_tran = pd.read_csv("twm_checking_tran.csv", sep=';')
twm_checking_acct = pd.read_csv("twm_checking_acct.csv", sep=';')
twm_accounts = pd.read_csv("twm_accounts.csv", sep=';')

In [5]:
find_replace = {"gender" : {"M": 1, "F": 0}}
twm_customer['gender'] = twm_customer['gender'].str.strip()
twm_customer.replace(find_replace, inplace=True)
twm_customer

,cust_id,income,age,years_with_bank,nbr_children,gender,marital_status,name_prefix,first_name,last_name,street_nbr,street_name,postal_code,city_name,state_code
0,1362691,26150,46,5,1,1,2,,Donald ...,Marek ...,8298,Second ...,89194,Las Vegas,NV
1,1362487,6605,71,1,0,1,2,,ChingDyi ...,Moussavi ...,10603,Daffodil ...,90159,Los Angeles,CA
2,1363160,18548,38,8,0,0,1,,Rosa ...,Johnston ...,8817,Figueroa ...,90024,Los Angeles,CA
3,1362752,47668,54,3,0,0,1,,Lisa ...,Martin ...,676,Humble ...,90172,Los Angeles,CA
4,1362548,44554,59,9,2,0,4,,Barbara ...,O'Malley ...,6578,C ...,10138,New York City,NY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,1363324,14795,36,6,1,0,4,,Lillian ...,Kaufman ...,9677,B ...,90016,Los Angeles,CA
743,1362895,26387,56,6,1,1,2,,Marty ...,McSherry ...,3227,Inspiration ...,10126,New York City,NY
744,1362569,61300,50,0,2,1,2,,Ken ...,Lawrence ...,6082,23rd ...,87194,Albuquerque,NM
745,1363364,15100,37,7,0,0,2,,Debbie ...,Runner ...,7851,H ...,35241,Birmingham,AL


In [6]:
test2=twm_customer.copy()
test2=test2.set_index('cust_id')
test3=twm_savings_acct.copy()
test3=test3.set_index('cust_id')


test2=test2.join(test3.ending_balance).rename({'ending_balance': 'saving'}, axis='columns')
test4=twm_credit_acct.copy()
test4=test4.set_index('cust_id')
test2=test2.join(test4.ending_balance).rename({'ending_balance': 'credit'}, axis='columns')
test5=twm_checking_acct.copy()
test5=test5.set_index('cust_id')
test2=test2.join(test5.ending_balance).rename({'ending_balance': 'checking'}, axis='columns')
test6=twm_credit_tran.groupby("cust_id").size().to_frame()
test2=test2.join(test6[0]).rename({0: 'num_credit_tran'}, axis='columns')

test7=twm_savings_tran.groupby("cust_id").size().to_frame()
test2=test2.join(test7[0]).rename({0: 'num_savings_tran'}, axis='columns')

test8=twm_checking_tran.groupby("cust_id").size().to_frame()
test2=test2.join(test8[0]).rename({0: 'num_checking_tran'}, axis='columns')


test9=twm_credit_tran.copy()
test9.tran_amt=test9.tran_amt.abs()
test9=test9.groupby('cust_id').tran_amt.sum().to_frame()  #re use test9
test2=test2.join(test9["tran_amt"]).rename({'tran_amt': 'CC_tran_amt'}, axis='columns')


test10=twm_checking_tran.copy()
test10.tran_amt=test10.tran_amt.abs()
test10=test10.groupby('cust_id').tran_amt.sum().to_frame()
test2=test2.join(test10["tran_amt"]).rename({'tran_amt': 'CH_tran_amt'}, axis='columns')

test11=twm_savings_tran.copy()
test11.tran_amt=test11.tran_amt.abs()
test11=test11.groupby('cust_id').tran_amt.sum().to_frame()
test2=test2.join(test11["tran_amt"]).rename({'tran_amt': 'SV_tran_amt'}, axis='columns')

test2['AVG_credit_amt']=test2['CC_tran_amt']/test2['num_credit_tran']
test2['AVG_savings_amt']=test2['SV_tran_amt']/test2['num_savings_tran']
test2['AVG_checking_amt']=test2['CH_tran_amt']/test2['num_checking_tran']


test2['Total_AVG_tran_amt']=(test2['CC_tran_amt'].fillna(0)+test2['SV_tran_amt'].fillna(0)+test2['CH_tran_amt'].fillna(0))/(test2['num_credit_tran'].fillna(0)+test2['num_savings_tran'].fillna(0)+test2['num_checking_tran'].fillna(0))

test2

,income,age,years_with_bank,nbr_children,gender,marital_status,name_prefix,first_name,last_name,street_nbr,...,num_credit_tran,num_savings_tran,num_checking_tran,CC_tran_amt,CH_tran_amt,SV_tran_amt,AVG_credit_amt,AVG_savings_amt,AVG_checking_amt,Total_AVG_tran_amt
cust_id,,,,,,,,,,,,,,,,,,,,,
1362691,26150,46,5,1,1,2,,Donald ...,Marek ...,8298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1362487,6605,71,1,0,1,2,,ChingDyi ...,Moussavi ...,10603,...,32.0,19.0,52.0,4773.21,3244.83,887.52,149.162813,46.711579,62.400577,86.461748
1363160,18548,38,8,0,0,1,,Rosa ...,Johnston ...,8817,...,68.0,48.0,NaN,12506.95,NaN,2662.22,183.925735,55.462917,NaN,130.768707
1362752,47668,54,3,0,0,1,,Lisa ...,Martin ...,676,...,15.0,12.0,93.0,6182.65,12086.71,9.07,412.176667,0.755833,129.964624,152.320250
1362548,44554,59,9,2,0,4,,Barbara ...,O'Malley ...,6578,...,22.0,12.0,99.0,15089.99,16817.43,3.78,685.908636,0.315000,169.873030,239.933835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363324,14795,36,6,1,0,4,,Lillian ...,Kaufman ...,9677,...,NaN,NaN,44.0,NaN,3093.48,NaN,NaN,NaN,70.306364,70.306364
1362895,26387,56,6,1,1,2,,Marty ...,McSherry ...,3227,...,27.0,28.0,NaN,3425.69,NaN,1410.77,126.877407,50.384643,NaN,87.935636
1362569,61300,50,0,2,1,2,,Ken ...,Lawrence ...,6082,...,28.0,NaN,99.0,9200.00,16219.81,NaN,328.571429,NaN,163.836465,200.155984


In [7]:
test2.columns

Index(['income', 'age', 'years_with_bank', 'nbr_children', 'gender',
       'marital_status', 'name_prefix', 'first_name', 'last_name',
       'street_nbr', 'street_name', 'postal_code', 'city_name', 'state_code',
       'saving', 'credit', 'checking', 'num_credit_tran', 'num_savings_tran',
       'num_checking_tran', 'CC_tran_amt', 'CH_tran_amt', 'SV_tran_amt',
       'AVG_credit_amt', 'AVG_savings_amt', 'AVG_checking_amt',
       'Total_AVG_tran_amt'],
      dtype='object')

In [8]:
pd.set_option('display.max_columns', None)


In [9]:
test2

,income,age,years_with_bank,nbr_children,gender,marital_status,name_prefix,first_name,last_name,street_nbr,street_name,postal_code,city_name,state_code,saving,credit,checking,num_credit_tran,num_savings_tran,num_checking_tran,CC_tran_amt,CH_tran_amt,SV_tran_amt,AVG_credit_amt,AVG_savings_amt,AVG_checking_amt,Total_AVG_tran_amt
cust_id,,,,,,,,,,,,,,,,,,,,,,,,,,,
1362691,26150,46,5,1,1,2,,Donald ...,Marek ...,8298,Second ...,89194,Las Vegas,NV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1362487,6605,71,1,0,1,2,,ChingDyi ...,Moussavi ...,10603,Daffodil ...,90159,Los Angeles,CA,12.89,1548.23,1401.21,32.0,19.0,52.0,4773.21,3244.83,887.52,149.162813,46.711579,62.400577,86.461748
1363160,18548,38,8,0,0,1,,Rosa ...,Johnston ...,8817,Figueroa ...,90024,Los Angeles,CA,122.54,286.69,NaN,68.0,48.0,NaN,12506.95,NaN,2662.22,183.925735,55.462917,NaN,130.768707
1362752,47668,54,3,0,0,1,,Lisa ...,Martin ...,676,Humble ...,90172,Los Angeles,CA,609.33,0.00,3.76,15.0,12.0,93.0,6182.65,12086.71,9.07,412.176667,0.755833,129.964624,152.320250
1362548,44554,59,9,2,0,4,,Barbara ...,O'Malley ...,6578,C ...,10138,New York City,NY,254.02,68.68,569.65,22.0,12.0,99.0,15089.99,16817.43,3.78,685.908636,0.315000,169.873030,239.933835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363324,14795,36,6,1,0,4,,Lillian ...,Kaufman ...,9677,B ...,90016,Los Angeles,CA,NaN,NaN,2325.37,NaN,NaN,44.0,NaN,3093.48,NaN,NaN,NaN,70.306364,70.306364
1362895,26387,56,6,1,1,2,,Marty ...,McSherry ...,3227,Inspiration ...,10126,New York City,NY,96.25,1300.00,NaN,27.0,28.0,NaN,3425.69,NaN,1410.77,126.877407,50.384643,NaN,87.935636
1362569,61300,50,0,2,1,2,,Ken ...,Lawrence ...,6082,23rd ...,87194,Albuquerque,NM,NaN,9200.00,46.48,28.0,NaN,99.0,9200.00,16219.81,NaN,328.571429,NaN,163.836465,200.155984
